# Tablas Principales

In [1]:
#CLIENTES_PORTAL_LEGALITAS 
clientes_portal_legalitas = spark.sql("SELECT * FROM LK_Gestion_Terceros_SLV.dbo_Clientes")

from pyspark.sql.functions import lit
clientes_portal_legalitas = clientes_portal_legalitas.withColumn("Servicio", lit(1))

# display(clientes_portal_legalitas)

StatementMeta(, f88df995-5ab3-4ee2-a11f-ea8ff55f4b69, 3, Finished, Available, Finished)

In [2]:
cuentas_crm = spark.sql("SELECT * FROM LK_Gestion_Terceros_SLV.dbo_CUENTAS where Borrado_stg = 0")

StatementMeta(, f88df995-5ab3-4ee2-a11f-ea8ff55f4b69, 4, Finished, Available, Finished)

In [3]:
#ADM_FINCAS_CRM
adm_fincas_crm = spark.sql("SELECT * FROM LK_Gestion_Terceros_SLV.dbo_ADM_FINCAS")
adm_fincas_crm = adm_fincas_crm.select('Cuenta_PK')
# display(adm_fincas_crm)

StatementMeta(, f88df995-5ab3-4ee2-a11f-ea8ff55f4b69, 5, Finished, Available, Finished)

In [4]:
#CLIENTES_INTRATIME
from pyspark.sql.functions import lit, to_date
df_clientes_intratime = spark.sql("SELECT * FROM LK_Gestion_Terceros_SLV.dbo_ClientesIntratime")
df_clientes_intratime = df_clientes_intratime.withColumn("FECHA_CARGA", to_date(lit("17-07-2024"), "dd-MM-yyyy"))

#CLIENTES_INTRATIME2
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_timestamp
df_clientes_intratime2 = spark.sql("SELECT * FROM LK_Gestion_Terceros_SLV.dbo_ClientesIntratime3")
df_clientes_intratime2 = df_clientes_intratime2.withColumn("FECHA_CARGA", df_clientes_intratime2["FECHA_CARGA"].cast("timestamp"))
df_clientes_intratime2 = df_clientes_intratime2.withColumn("FECHA_CARGA", to_date(df_clientes_intratime2["FECHA_CARGA"]))

df_clientes_intratime = df_clientes_intratime.unionByName(df_clientes_intratime2)

display(df_clientes_intratime)


StatementMeta(, f88df995-5ab3-4ee2-a11f-ea8ff55f4b69, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c6bcc4f2-caeb-45a9-8e90-b155e553b438)

In [5]:
#CLIENTES_INTRATIME_HOY
df_clientes_intratime_hoy = spark.sql("SELECT * FROM LK_Gestion_Terceros_SLV.dbo_ClientesIntratimeHoy2")

#Cambiar nombre de las columnas de df_clientes_intratime_hoy agregando un sufijo "HOY" a cada uno

sufijo_hoy = "_HOY"
columnas_actuales_hoy = df_clientes_intratime_hoy.columns
for columna in columnas_actuales_hoy:
    nuevo_nombre_hoy = columna + sufijo_hoy
    df_clientes_intratime_hoy = df_clientes_intratime_hoy.withColumnRenamed(columna, nuevo_nombre_hoy)

display(df_clientes_intratime_hoy)

StatementMeta(, f88df995-5ab3-4ee2-a11f-ea8ff55f4b69, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 275cc94a-a9f4-4029-a3e4-2fd306f8fbee)

In [6]:
from pyspark.sql.functions import col, when, lit, current_date
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, desc

"""Eliminamos duplicados del dataframe df_clientes_intratime ya que la canalización que trae la información a este lakehouse está con la función anexar
con la intención de no perder el registro de los clientes que se han dado de alta en algun momento"""

#df_clientes_intratime = df_clientes_intratime.dropDuplicates(['COMPANY_ID'])

#Cruzar dataframe df_clientes_intratime vs df_clientes_intratime_hoy para identificar los clientes que se han dado de baja
df_clientes_intratime = df_clientes_intratime.join(df_clientes_intratime_hoy.select("COMPANY_ID_HOY"), df_clientes_intratime["COMPANY_ID"] == df_clientes_intratime_hoy["COMPANY_ID_HOY"], "left")

#Crear columna fecha de baja (del día de hoy) para los clientes que no cruzan con la data de intratime del día de hoy
df_clientes_intratime = df_clientes_intratime.withColumn('FECHA_BAJA',when(col('COMPANY_ID_HOY').isNull(), col("FECHA_CARGA")).otherwise(lit(None)))

# Convertir la columna 'fecha' a tipo date
df_clientes_intratime = df_clientes_intratime.withColumn("FECHA_BAJA", col("FECHA_BAJA").cast("date"))

from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Definir la ventana para particionar por 'id' y ordenar por 'fecha' de forma descendente
windowSpec = Window.partitionBy("COMPANY_ID").orderBy(desc("FECHA_BAJA"))

# Agregar una columna 'row_number' que asigne un número de fila a cada partición
df_clientes_intratime = df_clientes_intratime.withColumn("row_number", row_number().over(windowSpec))

# Filtrar para conservar solo los registros con 'row_number' igual a 1
df_clientes_intratime = df_clientes_intratime.filter(col("row_number") == 1).drop("row_number")

#Cruzar vs dataframe cuentas_crm con la intención de incorporar la información de la cuenta_pk

df_clientes_intratime = df_clientes_intratime.join(cuentas_crm.select("Cuenta_PK","NIF"), df_clientes_intratime["COMPANY_CIF"] == cuentas_crm["NIF"], "left")

display(df_clientes_intratime)

StatementMeta(, f88df995-5ab3-4ee2-a11f-ea8ff55f4b69, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 848f8248-3a41-47ce-8828-df342df8d3cb)

In [7]:
#PRUEBA TEMPORAL
#df_prueba_temporal = df_clientes_intratime

StatementMeta(, f88df995-5ab3-4ee2-a11f-ea8ff55f4b69, 9, Finished, Available, Finished)

In [8]:
#PRUEBA TEMPORAL
#delta_table_name = "prueba_temporal"
#df_prueba_temporal.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(delta_table_name)
#print(f"DataFrame has been written to Delta table: {delta_table_name}")

StatementMeta(, f88df995-5ab3-4ee2-a11f-ea8ff55f4b69, 10, Finished, Available, Finished)

In [9]:
#Reordenamos y creamos columnas en el dataframe df_clientes_intratime de manera que tenga la misma estructura del dataframe clientes_portal_legalitas 

df_clientes_intratime = df_clientes_intratime.withColumn("Codigo", lit(None))
df_clientes_intratime = df_clientes_intratime.withColumn("Estado_EnvioEmail", lit(None))
df_clientes_intratime = df_clientes_intratime.withColumn("Activo", lit(None))
df_clientes_intratime = df_clientes_intratime.withColumn("Servicio", lit(2))

df_clientes_intratime = df_clientes_intratime.withColumnRenamed("COMPANY_ID","Id")
df_clientes_intratime = df_clientes_intratime.withColumnRenamed("COMPANY_CREATION_DATE","Fecha_Alta")
df_clientes_intratime = df_clientes_intratime.withColumnRenamed("FECHA_BAJA","Fecha_Baja")

df_clientes_intratime = df_clientes_intratime.select("Id","Cuenta_PK","Codigo","Estado_EnvioEmail","Fecha_Alta","Activo","Fecha_Baja","Servicio")

display(df_clientes_intratime)

StatementMeta(, f88df995-5ab3-4ee2-a11f-ea8ff55f4b69, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 5a6fad4d-b21d-461c-9b02-6475ea703a76)

In [10]:
#Hacer una unión de los dataframe clientes_portal_legalitas y df_clientes_intratime para formar el dataframe df_gestion_terceros

df_gestion_terceros = clientes_portal_legalitas.union(df_clientes_intratime)

#Cada tabla tenía su propio id_pk pero será necesario crear un nuevo id incremental para el nuevo dataframe resultante de la unión
from pyspark.sql.window import Window
from pyspark.sql.functions import monotonically_increasing_id, row_number

# Crear una ventana (window) para usar en la función row_number()
windowSpec = Window.orderBy(monotonically_increasing_id())

# Agregar una columna con el ID incremental comenzando desde 1
df_gestion_terceros = df_gestion_terceros.withColumn("ID_CLIENTE_PK", row_number().over(windowSpec))

df_gestion_terceros = df_gestion_terceros.select("ID_CLIENTE_PK","Cuenta_PK","Codigo","Estado_EnvioEmail","Fecha_Alta","Activo","Fecha_Baja","Servicio")

df_gestion_terceros = df_gestion_terceros.withColumnRenamed("ID_CLIENTE_PK","Id")

#display(df_gestion_terceros)

StatementMeta(, f88df995-5ab3-4ee2-a11f-ea8ff55f4b69, 12, Finished, Available, Finished)

# Cambio de Nombres de Columnas

In [11]:
#DF_GESTION_TERCEROS
df_gestion_terceros = df_gestion_terceros.withColumnRenamed("Id","Id_GESTION_TERCEROS")
df_gestion_terceros = df_gestion_terceros.withColumnRenamed("Cuenta_PK","Cuenta_PK_GESTION_TERCEROS")
df_gestion_terceros = df_gestion_terceros.withColumnRenamed("Codigo","Codigo_GESTION_TERCEROS")
df_gestion_terceros = df_gestion_terceros.withColumnRenamed("Estado_EnvioEmail","Estado_EnvioEmail_GESTION_TERCEROS")
df_gestion_terceros = df_gestion_terceros.withColumnRenamed("Fecha_Alta","Fecha_Alta_GESTION_TERCEROS")
df_gestion_terceros = df_gestion_terceros.withColumnRenamed("Activo","Activo_GESTION_TERCEROS")
df_gestion_terceros = df_gestion_terceros.withColumnRenamed("Fecha_Baja","Fecha_Baja_GESTION_TERCEROS")
df_gestion_terceros = df_gestion_terceros.withColumnRenamed("Servicio","Servicio_GESTION_TERCEROS")

#CUENTAS_CRM (agregamos un sufijo "CUENTAS_CRM a cada nombre de las columnas de la tabla")
sufijo_crm = "_CUENTAS_CRM"
columnas_actuales_crm = cuentas_crm.columns
for columna in columnas_actuales_crm:
    nuevo_combre_crm = columna + sufijo_crm
    cuentas_crm = cuentas_crm.withColumnRenamed(columna, nuevo_combre_crm)

#ADM_FINCAS_CRM (agregamos un sufijo "ADM_FINCAS a cada nombre de las columnas de la tabla")
sufijo = "_ADM_FINCAS"
columnas_actuales = adm_fincas_crm.columns
for columna in columnas_actuales:
    nuevo_combre = columna + sufijo
    adm_fincas_crm = adm_fincas_crm.withColumnRenamed(columna, nuevo_combre)

StatementMeta(, f88df995-5ab3-4ee2-a11f-ea8ff55f4b69, 13, Finished, Available, Finished)

## Agrupación de Tablas

In [12]:
# df_gestion_terceros y cuentas_crm
df_gestion_terceros = df_gestion_terceros.join(cuentas_crm, df_gestion_terceros["Cuenta_PK_GESTION_TERCEROS"] == cuentas_crm["Cuenta_PK_CUENTAS_CRM"], "left")
#display(df_gestion_terceros)

StatementMeta(, f88df995-5ab3-4ee2-a11f-ea8ff55f4b69, 14, Finished, Available, Finished)

In [13]:
# df_gestion_terceros y adm_fincas_crm
df_gestion_terceros = df_gestion_terceros.join(adm_fincas_crm, df_gestion_terceros["Cuenta_PK_GESTION_TERCEROS"] == adm_fincas_crm["Cuenta_PK_ADM_FINCAS"], "left")

#display(df_gestion_terceros)

StatementMeta(, f88df995-5ab3-4ee2-a11f-ea8ff55f4b69, 15, Finished, Available, Finished)

In [14]:
#Crear columna CON/SIN_EMPLEADOS en DF_GESTION_TERCEROS
from pyspark.sql.functions import when
from pyspark.sql.functions import col

df_gestion_terceros = df_gestion_terceros.withColumn('CON/SIN_EMPLEADOS',when(col('Codigo_GESTION_TERCEROS').isNull(), '2').otherwise('1'))

#Crear columna TIPO_CLIENTE en DF_GESTION_TERCEROS
df_gestion_terceros = df_gestion_terceros.withColumn('TIPO_CLIENTE',when(col('Cuenta_PK_ADM_FINCAS').isNull(), '1').otherwise('2'))
#display(df_gestion_terceros)

StatementMeta(, f88df995-5ab3-4ee2-a11f-ea8ff55f4b69, 16, Finished, Available, Finished)

## Crear Tablas Dimensionales

In [15]:
#tabla dim_sector_cliente

from pyspark.sql.types import StructType, StructField, StringType, IntegerType

#Definir esquema del dataframe

schema = StructType([
    StructField('ID_SECTOR_PK', IntegerType() , True),
    StructField('NOMBRE', StringType() , True)
    ])

#Crear dataframe

datos = [
     (0 , "PYME"),
     (1 , "AAFF"),
     (2 , "AAPP"),
     (3 , "CCPP"),
     (4 , "PNH")
]

dim_sector = spark.createDataFrame(datos , schema)              

display(dim_sector)

StatementMeta(, f88df995-5ab3-4ee2-a11f-ea8ff55f4b69, 17, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 3b6f5df8-4a2e-4173-9fa1-e39f4f8415f7)

In [16]:
#tabla dim_tipo_cliente

from pyspark.sql.types import StructType, StructField, StringType, IntegerType

#Definir esquema del dataframe

schema_tipo_cliente = StructType([
    StructField('ID_TIPO_PK', IntegerType() , True),
    StructField('NOMBRE', StringType() , True)
    ])

#Crear dataframe

datos_tipo_cliente = [
     (1 , "PYME"),
     (2 , "AAFF")
]

dim_tipo_cliente = spark.createDataFrame(datos_tipo_cliente , schema_tipo_cliente)              

display(dim_tipo_cliente)

StatementMeta(, f88df995-5ab3-4ee2-a11f-ea8ff55f4b69, 18, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 814c04e0-490f-4316-98c8-f0aa6327bf36)

In [17]:
#tabla dim_tipo_empleados

from pyspark.sql.types import StructType, StructField, StringType, IntegerType

#Definir esquema del dataframe

schema_tipo_empleados = StructType([
    StructField('ID_TIPO_PK', IntegerType() , True),
    StructField('NOMBRE', StringType() , True)
    ])

#Crear dataframe

datos_tipo_empleados = [
     (1 , "Con Empleados"),
     (2 , "Sin Empleados")
]

dim_tipo_empleados = spark.createDataFrame(datos_tipo_empleados , schema_tipo_empleados)              

display(dim_tipo_empleados)

StatementMeta(, f88df995-5ab3-4ee2-a11f-ea8ff55f4b69, 19, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 44daea7c-1802-4791-acf2-310770485bd6)

In [18]:
#tabla dim_clientes
from pyspark.sql.window import Window
from pyspark.sql.functions import monotonically_increasing_id, row_number

# Seleccionar la columna "NIF" del DataFrame df_gestion_terceros y obtener los valores únicos
valores_unicos_clientes = df_gestion_terceros.select ('NIF_CUENTAS_CRM','RazonSocial_CUENTAS_CRM')
valores_unicos_clientes = valores_unicos_clientes.filter(col('NIF_CUENTAS_CRM').isNotNull())
valores_unicos_clientes = valores_unicos_clientes.dropDuplicates(['NIF_CUENTAS_CRM'])

# Crear una ventana (window) para usar en la función row_number()
windowSpec = Window.orderBy(monotonically_increasing_id())

# Agregar una columna con el ID incremental comenzando desde 1
dim_clientes = valores_unicos_clientes.withColumn("ID_CLIENTE_PK", row_number().over(windowSpec))

# Reorganizar las columnas
dim_clientes = dim_clientes.select('ID_CLIENTE_PK','NIF_CUENTAS_CRM','RazonSocial_CUENTAS_CRM')

# Cambio de nombre a la columna
dim_clientes = dim_clientes.withColumnRenamed("NIF_CUENTAS_CRM","NIF_dim")
dim_clientes = dim_clientes.withColumnRenamed("RazonSocial_CUENTAS_CRM","RazonSocial_dim")

display(dim_clientes)

StatementMeta(, f88df995-5ab3-4ee2-a11f-ea8ff55f4b69, 20, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, db4f7795-ac3b-4d0b-bb4e-a01334214c2b)

In [19]:
#tabla dim_tipo_mov

from pyspark.sql.types import StructType, StructField, StringType, IntegerType

#Definir esquema del dataframe

schema_tipo_mov = StructType([
    StructField('ID_TIPO_PK', IntegerType() , True),
    StructField('NOMBRE', StringType() , True)
    ])

#Crear dataframe

datos_tipo_mov = [
     (0 , "Bajas"),
     (1 , "Altas")
]

dim_tipo_mov = spark.createDataFrame(datos_tipo_mov , schema_tipo_mov)              

#display(dim_tipo_mov)

StatementMeta(, f88df995-5ab3-4ee2-a11f-ea8ff55f4b69, 21, Finished, Available, Finished)

In [20]:
#tabla dim_servicio

from pyspark.sql.types import StructType, StructField, StringType, IntegerType

#Definir esquema del dataframe

schema_servicio = StructType([
    StructField('ID_SERVICIO_PK', IntegerType() , True),
    StructField('SERVICIO', StringType() , True)
    ])

#Crear dataframe

datos_servicio = [
     (1 , "Legalitas"),
     (2 , "Intratime")
]

dim_servicio = spark.createDataFrame(datos_servicio , schema_servicio)              

#display(dim_servicio)

StatementMeta(, f88df995-5ab3-4ee2-a11f-ea8ff55f4b69, 22, Finished, Available, Finished)

## Agrupación con Tablas Dimensionales

In [21]:
#df_gestion_terceros y dim_clientes

df_gestion_terceros = df_gestion_terceros.join(dim_clientes, df_gestion_terceros['NIF_CUENTAS_CRM'] == dim_clientes['NIF_dim'], "left") 

StatementMeta(, f88df995-5ab3-4ee2-a11f-ea8ff55f4b69, 23, Finished, Available, Finished)

## Creación de Tablas de Hechos (Altas, Bajas, Altas_Bajas)

En esta sección crearemos los dataframe de tablas de hechos de Altas, Bajas y Altas_Bajas que serán necesarios para construir y mostrar la evolución mes a mes del saldo de clientes. 

In [22]:
#Tabla de Hechos Altas

from pyspark.sql.functions import lit
from pyspark.sql.functions import month, date_format
from pyspark.sql.functions import year, col

#Tabla de Hechos Altas
gestion_terceros_altas = df_gestion_terceros

#Creamos columnas de TIPOMOV, FECHAMOV, NUMEROMESMOV, NOMBREMESMOV
gestion_terceros_altas = gestion_terceros_altas.withColumn("TIPO_MOV", lit(1))
gestion_terceros_altas = gestion_terceros_altas.withColumn("FECHA_MOV", col("Fecha_Alta_GESTION_TERCEROS"))
gestion_terceros_altas = gestion_terceros_altas.withColumn("NUMERO_MES_MOV", month(col("Fecha_Alta_GESTION_TERCEROS")))
gestion_terceros_altas = gestion_terceros_altas.withColumn("NOMBRE_MES_MOV", date_format(col("Fecha_Alta_GESTION_TERCEROS"),"MMMM"))
gestion_terceros_altas = gestion_terceros_altas.withColumn("ANIO_MOV" , year(col("Fecha_Alta_GESTION_TERCEROS")))

StatementMeta(, f88df995-5ab3-4ee2-a11f-ea8ff55f4b69, 24, Finished, Available, Finished)

In [23]:
#Tabla de Hechos Bajas

from pyspark.sql.functions import lit
from pyspark.sql.functions import month, date_format
from pyspark.sql.functions import year, col

#Tabla de Hechos Bajas
gestion_terceros_bajas = df_gestion_terceros.filter(col("Fecha_Baja_GESTION_TERCEROS").isNotNull())

#Creamos columnas de TIPOMOV, FECHAMOV, NUMEROMESMOV, NOMBREMESMOV
gestion_terceros_bajas = gestion_terceros_bajas.withColumn("TIPO_MOV", lit(0))
gestion_terceros_bajas = gestion_terceros_bajas.withColumn("FECHA_MOV", col("Fecha_Baja_GESTION_TERCEROS"))
gestion_terceros_bajas = gestion_terceros_bajas.withColumn("NUMERO_MES_MOV", month(col("Fecha_Baja_GESTION_TERCEROS")))
gestion_terceros_bajas = gestion_terceros_bajas.withColumn("NOMBRE_MES_MOV", date_format(col("Fecha_Baja_GESTION_TERCEROS"),"MMMM"))
gestion_terceros_bajas = gestion_terceros_bajas.withColumn("ANIO_MOV" , year(col("Fecha_Baja_GESTION_TERCEROS")))

StatementMeta(, f88df995-5ab3-4ee2-a11f-ea8ff55f4b69, 25, Finished, Available, Finished)

In [24]:
#Tabla de Hechos Altas_Bajas
gestion_terceros_altas_bajas = gestion_terceros_altas.union(gestion_terceros_bajas)

# Crear una ventana (window) para usar en la función row_number()
windowSpec = Window.orderBy(monotonically_increasing_id())

# Agregar una columna con el ID incremental comenzando desde 1
gestion_terceros_altas_bajas = gestion_terceros_altas_bajas.withColumn("ID_MOV_PK", row_number().over(windowSpec))

StatementMeta(, f88df995-5ab3-4ee2-a11f-ea8ff55f4b69, 26, Finished, Available, Finished)

## Inconvenientes

En este paso fue necesario eliminar campos de la tabla gestion terceros ya que producto de los join que se hicieron estaba presentando inconvenientes para crear la tabla delta

In [25]:
#Seleccionar Campos que se van a Necesitar de las Tablas 
df_gestion_terceros = df_gestion_terceros.select('Id_GESTION_TERCEROS','Cuenta_PK_GESTION_TERCEROS','Codigo_GESTION_TERCEROS','Estado_EnvioEmail_GESTION_TERCEROS','Fecha_Alta_GESTION_TERCEROS','Activo_GESTION_TERCEROS','Fecha_Baja_GESTION_TERCEROS','Cuenta_PK_CUENTAS_CRM','Colaborador_PK_CUENTAS_CRM','Actividad_PK_CUENTAS_CRM','CNAE_PK_CUENTAS_CRM','NIF_CUENTAS_CRM','RazonSocial_CUENTAS_CRM','Fecha_alta_CUENTAS_CRM','Fecha_modificacion_CUENTAS_CRM','Asignado_CUENTAS_CRM','Direccion_Facturacion_CUENTAS_CRM','CP_Facturacion_CUENTAS_CRM','Poblacion_Facturacion_PK_CUENTAS_CRM','Provincia_Facturacion_PK_CUENTAS_CRM','Empleados_CUENTAS_CRM','Categoria_CUENTAS_CRM','referencia_CUENTAS_CRM','sector_cliente_CUENTAS_CRM','Fecha_modificacion_stg_CUENTAS_CRM','CON/SIN_EMPLEADOS','TIPO_CLIENTE','ID_CLIENTE_PK','NIF_dim','RazonSocial_dim',"Servicio_GESTION_TERCEROS")
gestion_terceros_altas = gestion_terceros_altas.select('Id_GESTION_TERCEROS','Cuenta_PK_GESTION_TERCEROS','Codigo_GESTION_TERCEROS','Estado_EnvioEmail_GESTION_TERCEROS','Fecha_Alta_GESTION_TERCEROS','Activo_GESTION_TERCEROS','Fecha_Baja_GESTION_TERCEROS','Cuenta_PK_CUENTAS_CRM','Colaborador_PK_CUENTAS_CRM','Actividad_PK_CUENTAS_CRM','CNAE_PK_CUENTAS_CRM','NIF_CUENTAS_CRM','RazonSocial_CUENTAS_CRM','Fecha_alta_CUENTAS_CRM','Fecha_modificacion_CUENTAS_CRM','Asignado_CUENTAS_CRM','Direccion_Facturacion_CUENTAS_CRM','CP_Facturacion_CUENTAS_CRM','Poblacion_Facturacion_PK_CUENTAS_CRM','Provincia_Facturacion_PK_CUENTAS_CRM','Empleados_CUENTAS_CRM','Categoria_CUENTAS_CRM','referencia_CUENTAS_CRM','sector_cliente_CUENTAS_CRM','Fecha_modificacion_stg_CUENTAS_CRM','CON/SIN_EMPLEADOS','TIPO_CLIENTE','ID_CLIENTE_PK','NIF_dim','RazonSocial_dim',"TIPO_MOV","FECHA_MOV","NUMERO_MES_MOV","NOMBRE_MES_MOV","ANIO_MOV","Servicio_GESTION_TERCEROS")
gestion_terceros_bajas = gestion_terceros_bajas.select('Id_GESTION_TERCEROS','Cuenta_PK_GESTION_TERCEROS','Codigo_GESTION_TERCEROS','Estado_EnvioEmail_GESTION_TERCEROS','Fecha_Alta_GESTION_TERCEROS','Activo_GESTION_TERCEROS','Fecha_Baja_GESTION_TERCEROS','Cuenta_PK_CUENTAS_CRM','Colaborador_PK_CUENTAS_CRM','Actividad_PK_CUENTAS_CRM','CNAE_PK_CUENTAS_CRM','NIF_CUENTAS_CRM','RazonSocial_CUENTAS_CRM','Fecha_alta_CUENTAS_CRM','Fecha_modificacion_CUENTAS_CRM','Asignado_CUENTAS_CRM','Direccion_Facturacion_CUENTAS_CRM','CP_Facturacion_CUENTAS_CRM','Poblacion_Facturacion_PK_CUENTAS_CRM','Provincia_Facturacion_PK_CUENTAS_CRM','Empleados_CUENTAS_CRM','Categoria_CUENTAS_CRM','referencia_CUENTAS_CRM','sector_cliente_CUENTAS_CRM','Fecha_modificacion_stg_CUENTAS_CRM','CON/SIN_EMPLEADOS','TIPO_CLIENTE','ID_CLIENTE_PK','NIF_dim','RazonSocial_dim',"TIPO_MOV","FECHA_MOV","NUMERO_MES_MOV","NOMBRE_MES_MOV","ANIO_MOV","Servicio_GESTION_TERCEROS")
gestion_terceros_altas_bajas = gestion_terceros_altas_bajas.select('ID_MOV_PK','Id_GESTION_TERCEROS','Cuenta_PK_GESTION_TERCEROS','Codigo_GESTION_TERCEROS','Estado_EnvioEmail_GESTION_TERCEROS','Fecha_Alta_GESTION_TERCEROS','Activo_GESTION_TERCEROS','Fecha_Baja_GESTION_TERCEROS','Cuenta_PK_CUENTAS_CRM','Colaborador_PK_CUENTAS_CRM','Actividad_PK_CUENTAS_CRM','CNAE_PK_CUENTAS_CRM','NIF_CUENTAS_CRM','RazonSocial_CUENTAS_CRM','Fecha_alta_CUENTAS_CRM','Fecha_modificacion_CUENTAS_CRM','Asignado_CUENTAS_CRM','Direccion_Facturacion_CUENTAS_CRM','CP_Facturacion_CUENTAS_CRM','Poblacion_Facturacion_PK_CUENTAS_CRM','Provincia_Facturacion_PK_CUENTAS_CRM','Empleados_CUENTAS_CRM','Categoria_CUENTAS_CRM','referencia_CUENTAS_CRM','sector_cliente_CUENTAS_CRM','Fecha_modificacion_stg_CUENTAS_CRM','CON/SIN_EMPLEADOS','TIPO_CLIENTE','ID_CLIENTE_PK','NIF_dim','RazonSocial_dim',"TIPO_MOV","FECHA_MOV","NUMERO_MES_MOV","NOMBRE_MES_MOV","ANIO_MOV","Servicio_GESTION_TERCEROS")

StatementMeta(, f88df995-5ab3-4ee2-a11f-ea8ff55f4b69, 27, Finished, Available, Finished)

## Creación Tablas Delta

In [26]:
# Tabla Delta FACT_ALTAS_BAJAS

delta_table_name = "slv_fact_altas_bajas"
gestion_terceros_altas_bajas.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(delta_table_name)
print(f"DataFrame has been written to Delta table: {delta_table_name}")

StatementMeta(, f88df995-5ab3-4ee2-a11f-ea8ff55f4b69, 28, Finished, Available, Finished)

DataFrame has been written to Delta table: slv_fact_altas_bajas


In [27]:
# Tabla Delta FACT_ALTAS

delta_table_name = "slv_fact_altas"
gestion_terceros_altas.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(delta_table_name)
print(f"DataFrame has been written to Delta table: {delta_table_name}")

StatementMeta(, f88df995-5ab3-4ee2-a11f-ea8ff55f4b69, 29, Finished, Available, Finished)

DataFrame has been written to Delta table: slv_fact_altas


In [28]:
# Tabla Delta FACT_BAJAS

delta_table_name = "slv_fact_bajas"
gestion_terceros_bajas.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(delta_table_name)
print(f"DataFrame has been written to Delta table: {delta_table_name}")

StatementMeta(, f88df995-5ab3-4ee2-a11f-ea8ff55f4b69, 30, Finished, Available, Finished)

DataFrame has been written to Delta table: slv_fact_bajas


In [29]:
# Tabla Delta GESTION_TERCEROS

delta_table_name = "slv_gestion_terceros"
df_gestion_terceros.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(delta_table_name)
print(f"DataFrame has been written to Delta table: {delta_table_name}")

StatementMeta(, f88df995-5ab3-4ee2-a11f-ea8ff55f4b69, 31, Finished, Available, Finished)

DataFrame has been written to Delta table: slv_gestion_terceros


In [30]:
# Tabla Delta DIM_SECTOR

delta_table_name = "slv_dim_sector"
dim_sector.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(delta_table_name)
print(f"DataFrame has been written to Delta table: {delta_table_name}")

StatementMeta(, f88df995-5ab3-4ee2-a11f-ea8ff55f4b69, 32, Finished, Available, Finished)

DataFrame has been written to Delta table: slv_dim_sector


In [31]:
# Tabla Delta DIM_TIPO_CLIENTE

delta_table_name = "slv_dim_tipo_cliente"
dim_tipo_cliente.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(delta_table_name)
print(f"DataFrame has been written to Delta table: {delta_table_name}")

StatementMeta(, f88df995-5ab3-4ee2-a11f-ea8ff55f4b69, 33, Finished, Available, Finished)

DataFrame has been written to Delta table: slv_dim_tipo_cliente


In [32]:
# Tabla Delta DIM_TIPO_CON/SINEMPLEADOS

delta_table_name = "slv_dim_con_sin_empleados"
dim_tipo_empleados.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(delta_table_name)
print(f"DataFrame has been written to Delta table: {delta_table_name}")

StatementMeta(, f88df995-5ab3-4ee2-a11f-ea8ff55f4b69, 34, Finished, Available, Finished)

DataFrame has been written to Delta table: slv_dim_con_sin_empleados


In [33]:
# Tabla Delta DIM_CLIENTES

delta_table_name = "slv_dim_clientes"
dim_clientes.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(delta_table_name)
print(f"DataFrame has been written to Delta table: {delta_table_name}")

StatementMeta(, f88df995-5ab3-4ee2-a11f-ea8ff55f4b69, 35, Finished, Available, Finished)

DataFrame has been written to Delta table: slv_dim_clientes


In [34]:
# Tabla Delta DIM_TIPO_MOV

delta_table_name = "slv_dim_tipo_mov"
dim_tipo_mov.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(delta_table_name)
print(f"DataFrame has been written to Delta table: {delta_table_name}")

StatementMeta(, f88df995-5ab3-4ee2-a11f-ea8ff55f4b69, 36, Finished, Available, Finished)

DataFrame has been written to Delta table: slv_dim_tipo_mov


In [35]:
# Tabla Delta DIM_SERVICIO

delta_table_name = "slv_dim_servicio"
dim_servicio.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(delta_table_name)
print(f"DataFrame has been written to Delta table: {delta_table_name}")

StatementMeta(, f88df995-5ab3-4ee2-a11f-ea8ff55f4b69, 37, Finished, Available, Finished)

DataFrame has been written to Delta table: slv_dim_servicio
